In [1]:
import subprocess as sp
import pandas as pd
from Bio import PDB
import shlex
import time

In [2]:
# Initialize loading class
load_struct = PDB.PDBList()

In [ ]:
# Parse command to load pdb index file
command = shlex.split('wget ftp://ftp.wwpdb.org/pub/pdb/derived_data/index/entries.idx')
# Run command
sp.check_output(command, universal_newlines=True)

In [9]:
# Delete junk --lines in file, make header tab-separated as whole other file
with open('entries.idx', 'r') as source, open('pdb_index', 'w') as target:
    for ind, line in enumerate(source):
        if ind == 0:
            line = line.replace(', ', '\t')
        if not line.startswith('-'):
            target.write(line)

In [2]:
# Create dataframe from full file with pdb index
with open('pdb_index', 'r') as source:
    data = pd.read_csv(source, sep='\t')

print(data.shape, data.columns)
data.head()

(134436, 8) Index(['IDCODE', 'HEADER', 'ACCESSION DATE', 'COMPOUND', 'SOURCE',
       'AUTHOR LIST', 'RESOLUTION', 'EXPERIMENT TYPE (IF NOT X-RAY)'],
      dtype='object')


,IDCODE,HEADER,ACCESSION DATE,COMPOUND,SOURCE,AUTHOR LIST,RESOLUTION,EXPERIMENT TYPE (IF NOT X-RAY)
0,100D,DNA-RNA HYBRID,12/05/94,CRYSTAL STRUCTURE OF THE HIGHLY DISTORTED CHIM...,NaN,"Ban, C., Ramakrishnan, B., Sundaralingam, M.",1.9,X-RAY DIFFRACTION
1,101D,DNA,12/14/94,REFINEMENT OF NETROPSIN BOUND TO DNA: BIAS AND...,NaN,"Goodsell, D.S., Kopka, M.L., Dickerson, R.E.",2.25,X-RAY DIFFRACTION
2,101M,OXYGEN TRANSPORT,12/13/97,SPERM WHALE MYOGLOBIN F46V N-BUTYL ISOCYANIDE ...,PHYSETER CATODON,"Smith, R.D., Olson, J.S., Phillips Jr., G.N.",2.07,X-RAY DIFFRACTION
3,102D,DNA,12/15/94,SEQUENCE-DEPENDENT DRUG BINDING TO THE MINOR G...,NaN,"Nunn, C.M., Neidle, S.",2.2,X-RAY DIFFRACTION
4,102L,HYDROLASE(O-GLYCOSYL),09/29/92,HOW AMINO-ACID INSERTIONS ARE ALLOWED IN AN AL...,ENTEROBACTERIA PHAGE T4,"Heinz, D.W., Matthews, B.W.",1.74,X-RAY DIFFRACTION


In [3]:
# Checking data
print(data.isnull().any(), data.shape)

# Drop files with empty header
data.dropna(subset=['HEADER'], inplace=True)
print(data.shape)

# # Filter subset of data with RNA in header
mask = data['HEADER'].str.contains('RNA')
rna = data[mask]


IDCODE                            False
HEADER                             True
ACCESSION DATE                    False
COMPOUND                          False
SOURCE                             True
AUTHOR LIST                       False
RESOLUTION                        False
EXPERIMENT TYPE (IF NOT X-RAY)    False
dtype: bool (134436, 8)
(134434, 8)


In [7]:
# Create list with PDB ids of files with RNA
rna_ids = rna['IDCODE'].unique().tolist()
rna_length = len(rna_ids)
rna_length, rna.head(), rna_ids

(3695,     IDCODE          HEADER ACCESSION DATE  \
 0     100D  DNA-RNA HYBRID       12/05/94   
 9     104D  DNA-RNA HYBRID       12/16/94   
 60    124D  DNA-RNA HYBRID       05/07/93   
 129   157D             RNA       02/01/94   
 138   161D  DNA-RNA HYBRID       02/10/94   
 
                                               COMPOUND SOURCE  \
 0    CRYSTAL STRUCTURE OF THE HIGHLY DISTORTED CHIM...    NaN   
 9    DNA DUPLEXES FLANKED BY HYBRID DUPLEXES: THE S...    NaN   
 60   STRUCTURE OF A DNA:RNA HYBRID DUPLEX: WHY RNAS...    NaN   
 129  CRYSTAL AND MOLECULAR STRUCTURE OF R(CGCGAAUUA...    NaN   
 138  A SINGLE 2'-HYDROXYL GROUP CONVERTS B-DNA TO A...    NaN   
 
                                            AUTHOR LIST RESOLUTION  \
 0         Ban, C., Ramakrishnan, B., Sundaralingam, M.        1.9   
 9                     Zhu, L., Salazar, M., Reid, B.R.        NOT   
 60             Fedoroff, O.Y., Salazar, M., Reid, B.R.        NOT   
 129  Leonard, G.A., McAuley-Hecht, K.

In [8]:
with open('rna_ids', 'w') as file:
    for entry in rna_ids:
        file.write('{}\n'.format(entry))

In [19]:
# Load 1 pdb file from RNA list, wait 30 seconds
for ind, file in enumerate(rna_ids, 1):
    load_struct.retrieve_pdb_file(file, file_format='pdb',pdir='/home/arleg/PycharmProjects/Bioinformatics/RNA/pdb')
    print("{} is loaded, {} from {}".format(file, ind, rna_length))
    time.sleep(30)


# Load 1 mmCIF file from RNA list, wait 30 seconds
# for ind, r_id in enumerate(rna_ids, 1):
#     load_struct.retrieve_pdb_file(r_id, file_format='mmCif',pdir='/home/arleg/PycharmProjects/Bioinformatics/RNA/CIF')
#     print("{} is loaded, {} from {}".format(r_id, ind, rna_length))
#     time.sleep(30)

100D is loaded, 1 from 3695
104D is loaded, 2 from 3695
124D is loaded, 3 from 3695
157D is loaded, 4 from 3695
161D is loaded, 5 from 3695
165D is loaded, 6 from 3695
168D is loaded, 7 from 3695
169D is loaded, 8 from 3695
176D is loaded, 9 from 3695
17RA is loaded, 10 from 3695
1A1T is loaded, 11 from 3695
1A34 is loaded, 12 from 3695
1A3M is loaded, 13 from 3695
1A4D is loaded, 14 from 3695
1A4T is loaded, 15 from 3695
1A51 is loaded, 16 from 3695
1A60 is loaded, 17 from 3695
1A8H is loaded, 18 from 3695
1A9L is loaded, 19 from 3695
1A9N is loaded, 20 from 3695
1AC3 is loaded, 21 from 3695
1ADJ is loaded, 22 from 3695
1ADY is loaded, 23 from 3695
1AFX is loaded, 24 from 3695
1AIL is loaded, 25 from 3695
1AJF is loaded, 26 from 3695
1AJL is loaded, 27 from 3695
1AJT is loaded, 28 from 3695
1AJU is loaded, 29 from 3695
1AKX is loaded, 30 from 3695
1AL5 is loaded, 31 from 3695
1AM0 is loaded, 32 from 3695
1ANR is loaded, 33 from 3695
1AP8 is loaded, 34 from 3695
1AQ3 is loaded, 35 from

1EHZ is loaded, 127 from 3695
1EI2 is loaded, 128 from 3695
1EIY is loaded, 129 from 3695
1EJZ is loaded, 130 from 3695
1EKA is loaded, 131 from 3695
1EKD is loaded, 132 from 3695
1EKZ is loaded, 133 from 3695
1ELH is loaded, 134 from 3695
1EQQ is loaded, 135 from 3695
1ESH is loaded, 136 from 3695
1ESY is loaded, 137 from 3695
1ET4 is loaded, 138 from 3695
1ETF is loaded, 139 from 3695
1ETG is loaded, 140 from 3695
1EUQ is loaded, 141 from 3695
1EUY is loaded, 142 from 3695
1EVP is loaded, 143 from 3695
1EVV is loaded, 144 from 3695
1EXD is loaded, 145 from 3695
1EXY is loaded, 146 from 3695
1F1T is loaded, 147 from 3695
1F27 is loaded, 148 from 3695
1F5G is loaded, 149 from 3695
1F5H is loaded, 150 from 3695
1F5U is loaded, 151 from 3695
1F6U is loaded, 152 from 3695
1F6X is loaded, 153 from 3695
1F6Z is loaded, 154 from 3695
1F78 is loaded, 155 from 3695
1F79 is loaded, 156 from 3695
1F7F is loaded, 157 from 3695
1F7G is loaded, 158 from 3695
1F7H is loaded, 159 from 3695
1F7I is lo

1I9V is loaded, 251 from 3695
1I9X is loaded, 252 from 3695
1IB2 is loaded, 253 from 3695
1ICG is loaded, 254 from 3695
1ID9 is loaded, 255 from 3695
1IDV is loaded, 256 from 3695
1IDW is loaded, 257 from 3695
1IE1 is loaded, 258 from 3695
1IE2 is loaded, 259 from 3695
1IFW is loaded, 260 from 3695
1IHA is loaded, 261 from 3695
1IK1 is loaded, 262 from 3695
1IK5 is loaded, 263 from 3695
1IKD is loaded, 264 from 3695
1IL2 is loaded, 265 from 3695
1ILE is loaded, 266 from 3695
1ILY is loaded, 267 from 3695
1IMU is loaded, 268 from 3695
1IPB is loaded, 269 from 3695
1IPC is loaded, 270 from 3695
1IQ4 is loaded, 271 from 3695
1IQT is loaded, 272 from 3695
1IVS is loaded, 273 from 3695
1J1H is loaded, 274 from 3695
1J1U is loaded, 275 from 3695
1J2B is loaded, 276 from 3695
1J42 is loaded, 277 from 3695
1J4Y is loaded, 278 from 3695
1J6S is loaded, 279 from 3695
1J7T is loaded, 280 from 3695
1J8G is loaded, 281 from 3695
1J9H is loaded, 282 from 3695
1JB8 is loaded, 283 from 3695
1JBR is lo

1M5V is loaded, 376 from 3695
1M82 is loaded, 377 from 3695
1M8V is loaded, 378 from 3695
1M8W is loaded, 379 from 3695
1M8X is loaded, 380 from 3695
1M8Y is loaded, 381 from 3695
1M8Z is loaded, 382 from 3695
1MDG is loaded, 383 from 3695
1ME0 is loaded, 384 from 3695
1ME1 is loaded, 385 from 3695
1MEA is loaded, 386 from 3695
1MED is loaded, 387 from 3695
1MFJ is loaded, 388 from 3695
1MFK is loaded, 389 from 3695
1MFQ is loaded, 390 from 3695
1MFY is loaded, 391 from 3695
1MGQ is loaded, 392 from 3695
1MHK is loaded, 393 from 3695
1MHN is loaded, 394 from 3695
1MIS is loaded, 395 from 3695
1MNB is loaded, 396 from 3695
1MNX is loaded, 397 from 3695
1MP1 is loaded, 398 from 3695
1MSW is loaded, 399 from 3695
1MSY is loaded, 400 from 3695
1MT4 is loaded, 401 from 3695
1MUV is loaded, 402 from 3695
1MV1 is loaded, 403 from 3695
1MV2 is loaded, 404 from 3695
1MV6 is loaded, 405 from 3695
1MWG is loaded, 406 from 3695
1MWL is loaded, 407 from 3695
1MY9 is loaded, 408 from 3695
1N1H is lo

1QC8 is loaded, 500 from 3695
1QCU is loaded, 501 from 3695
1QD3 is loaded, 502 from 3695
1QES is loaded, 503 from 3695
1QET is loaded, 504 from 3695
1QF6 is loaded, 505 from 3695
1QFQ is loaded, 506 from 3695
1QKL is loaded, 507 from 3695
1QRS is loaded, 508 from 3695
1QRT is loaded, 509 from 3695
1QRU is loaded, 510 from 3695
1QTQ is loaded, 511 from 3695
1QU2 is loaded, 512 from 3695
1QU3 is loaded, 513 from 3695
1QWA is loaded, 514 from 3695
1QWB is loaded, 515 from 3695
1QZA is loaded, 516 from 3695
1QZB is loaded, 517 from 3695
1QZC is loaded, 518 from 3695
1QZW is loaded, 519 from 3695
1R2P is loaded, 520 from 3695
1R2W is loaded, 521 from 3695
1R2X is loaded, 522 from 3695
1R3E is loaded, 523 from 3695
1R3O is loaded, 524 from 3695
1R3X is loaded, 525 from 3695
1R4H is loaded, 526 from 3695
1R4K is loaded, 527 from 3695
1R7W is loaded, 528 from 3695
1R7Z is loaded, 529 from 3695
1R9F is loaded, 530 from 3695
1R9S is loaded, 531 from 3695
1R9T is loaded, 532 from 3695
1RAR is lo

...

5AWH is loaded, 3200 from 3695
5AXM is loaded, 3201 from 3695
5AXN is loaded, 3202 from 3695
5AXW is loaded, 3203 from 3695
5AY2 is loaded, 3204 from 3695
5AY3 is loaded, 3205 from 3695
5AY4 is loaded, 3206 from 3695
5B2O is loaded, 3207 from 3695
5B2P is loaded, 3208 from 3695
5B2Q is loaded, 3209 from 3695
5B2R is loaded, 3210 from 3695
5B2S is loaded, 3211 from 3695
5B2T is loaded, 3212 from 3695
5B43 is loaded, 3213 from 3695
5B88 is loaded, 3214 from 3695
5BJO is loaded, 3215 from 3695
5BJP is loaded, 3216 from 3695
5BS3 is loaded, 3217 from 3695
5BTM is loaded, 3218 from 3695
5BTP is loaded, 3219 from 3695
5BUD is loaded, 3220 from 3695
5BWS is loaded, 3221 from 3695
5BXK is loaded, 3222 from 3695
5C0V is loaded, 3223 from 3695
5C0W is loaded, 3224 from 3695
5C0X is loaded, 3225 from 3695
5C3E is loaded, 3226 from 3695
5C44 is loaded, 3227 from 3695
5C45 is loaded, 3228 from 3695
5C4A is loaded, 3229 from 3695
5C4J is loaded, 3230 from 3695
5C4X is loaded, 3231 from 3695
5C5W is 

5EV3 is loaded, 3323 from 3695
5EV4 is loaded, 3324 from 3695
5EW4 is loaded, 3325 from 3695
5EW7 is loaded, 3326 from 3695
5EWR is loaded, 3327 from 3695
5EX7 is loaded, 3328 from 3695
5F0Q is loaded, 3329 from 3695
5F0S is loaded, 3330 from 3695
5F3O is loaded, 3331 from 3695
5F3P is loaded, 3332 from 3695
5F3Q is loaded, 3333 from 3695
5F5F is loaded, 3334 from 3695
5F5H is loaded, 3335 from 3695
5F7V is loaded, 3336 from 3695
5F8G is loaded, 3337 from 3695
5F8H is loaded, 3338 from 3695
5F8I is loaded, 3339 from 3695
5F8J is loaded, 3340 from 3695
5F8L is loaded, 3341 from 3695
5F8M is loaded, 3342 from 3695
5F8N is loaded, 3343 from 3695
5F9R is loaded, 3344 from 3695
5FJ0 is loaded, 3345 from 3695
5FJ1 is loaded, 3346 from 3695
5FJC is loaded, 3347 from 3695
5FK1 is loaded, 3348 from 3695
5FK2 is loaded, 3349 from 3695
5FK3 is loaded, 3350 from 3695
5FK4 is loaded, 3351 from 3695
5FK5 is loaded, 3352 from 3695
5FK6 is loaded, 3353 from 3695
5FKD is loaded, 3354 from 3695
5FKE is 

5JS1 is loaded, 3445 from 3695
5JS2 is loaded, 3446 from 3695
5JUN is loaded, 3447 from 3695
5JUR is loaded, 3448 from 3695
5K36 is loaded, 3449 from 3695
5K77 is loaded, 3450 from 3695
5K78 is loaded, 3451 from 3695
5K7C is loaded, 3452 from 3695
5K7D is loaded, 3453 from 3695
5K7E is loaded, 3454 from 3695
5K8H is loaded, 3455 from 3695
5KAL is loaded, 3456 from 3695
5KH8 is loaded, 3457 from 3695
5KI6 is loaded, 3458 from 3695
5KK5 is loaded, 3459 from 3695
5KL1 is loaded, 3460 from 3695
5KL8 is loaded, 3461 from 3695
5KLA is loaded, 3462 from 3695
5KMZ is loaded, 3463 from 3695
5KNW is loaded, 3464 from 3695
5KPY is loaded, 3465 from 3695
5KQE is loaded, 3466 from 3695
5KRG is loaded, 3467 from 3695
5KTJ is loaded, 3468 from 3695
5KVJ is loaded, 3469 from 3695
5KX9 is loaded, 3470 from 3695
5KXF is loaded, 3471 from 3695
5KZK is loaded, 3472 from 3695
5L00 is loaded, 3473 from 3695
5L1Z is loaded, 3474 from 3695
5L2L is loaded, 3475 from 3695
5L7Z is loaded, 3476 from 3695
5L80 is 

5TSN is loaded, 3567 from 3695
5U34 is loaded, 3568 from 3695
5U3G is loaded, 3569 from 3695
5U6H is loaded, 3570 from 3695
5U6L is loaded, 3571 from 3695
5U9B is loaded, 3572 from 3695
5UBA is loaded, 3573 from 3695
5UD5 is loaded, 3574 from 3695
5UDI is loaded, 3575 from 3695
5UDJ is loaded, 3576 from 3695
5UDK is loaded, 3577 from 3695
5UDL is loaded, 3578 from 3695
5UDZ is loaded, 3579 from 3695
5UED is loaded, 3580 from 3695
5UEE is loaded, 3581 from 3695
5UEF is loaded, 3582 from 3695
5UEG is loaded, 3583 from 3695
5UF3 is loaded, 3584 from 3695
5UH5 is loaded, 3585 from 3695
5UH8 is loaded, 3586 from 3695
5UH9 is loaded, 3587 from 3695
5UHB is loaded, 3588 from 3695
5UHC is loaded, 3589 from 3695
5UHD is loaded, 3590 from 3695
5UHE is loaded, 3591 from 3695
5UHF is loaded, 3592 from 3695
5UHG is loaded, 3593 from 3695
5UJ2 is loaded, 3594 from 3695
5UK4 is loaded, 3595 from 3695
5UKB is loaded, 3596 from 3695
5UNE is loaded, 3597 from 3695
5US2 is loaded, 3598 from 3695
5USA is 

7MSF is loaded, 3690 from 3695
7RAT is loaded, 3691 from 3695
8DRH is loaded, 3692 from 3695
8PSH is loaded, 3693 from 3695
8RAT is loaded, 3694 from 3695
9RAT is loaded, 3695 from 3695
